# Time travel operations in Hopsworks Feature Store

In this notebook we will introduce time travel operations in Hopsworks Feature Store (HSFS). Currently HSFS supports Apache Hudi (http://hudi.apache.org/) a storage abstraction/library for doing **incremental** data ingestion to a Hopsworks Feature Store.

## Background

### Motivation

Traditional ETL typically involves taking a snapshot of a production database and doing a full load into a data lake (typically stored on a distributed file system). Using the snapshot approach for ETL is simple since the snapshot is immutable and can be loaded as an atomic unit into the data lake. However, the con of taking this approach to doing data ingestion is that it is *slow*. Even if just a single record have been updated since the last data ingestion, the entire table has to be re-written. If you are working with Big Data (TB or PB size datasets) then this will introduce significant *data latency* and *wasted resources* (majority of the writes when ingesting the snapshot is redundant as most of the records have not been updated since the last ETL step). 

This motivates the use-case for **incremental** data ingestion. Incremental data ingestion means that only deltas/changelogs since the last ingestion are inserted. With incremental processing, you process data in *mini-batches* and run the spark job frequently. The incremental model makes better use of resources and makes it easier to do complex processing and joins.

In addition data is rarely immutable in practice. A bank transaction might be reverted, a customer might change his or her home adress, and a customer review might be updated, to give a few examples. This is where Hudi comes into the picture. Hudi stands for `Hadoop Upserts anD Incrementals` and brings two new primitives for data engineering on distributed file systems (in addition to append/read):

- `Upsert`: the ability to do insertions (appends) and updates efficiently. 
- `Incremental reads`: the ability to read datasets incrementally using the notion of "commits".

### Importing necessary libraries 

In [1]:
import hsfs
import datetime
from pyspark.sql import DataFrame, Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime

connection = hsfs.connection()
# get a reference to the feature store, you can access also shared feature stores by providing the feature store name
fs = connection.get_feature_store();

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
17,application_1624399475793_0002,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

In [2]:
prodocut_views_schema = StructType([
  StructField("event_timestamp", LongType(), True),
  StructField("event_time", StringType(), True),
  StructField("hour", StringType(), True),
  StructField("user_id", LongType(), True), 
  StructField("user_session", StringType(), True),      
  StructField("event_type", StringType(), True),     
  StructField("product_id", LongType(), True)])

producs_schema = StructType([
  StructField("product_id", IntegerType(), True),
  StructField("category", StringType(), True),
  StructField("sub_category", StringType(), True),   
  StructField("brand", StringType(), True),
  StructField("price", FloatType(), True)    
])

### Create spark dataframes for each Feature groups

In [3]:
prodocut_views_insert = [
    Row(1624397734,"2021-06-22 23:35:34", "2021-06-22-23", 535871217, "c6bd7419-2748-4c56-95b4-8cec9ff8b80d", "view", 1000001),
    Row(1624398017,"2021-06-22 23:40:17", "2021-06-22-23", 535871217, "c6bd7419-2748-4c56-95b4-8cec9ff8b80d", "view", 1000006),
    Row(1624398346,"2021-06-22 23:45:46", "2021-06-22-23", 524325294, "0b74a829-f9d7-4654-b5b0-35bc9822c238", "view", 1000001),
    Row(1624398650,"2021-06-22 23:50:50", "2021-06-22-23", 524325294, "0b74a829-f9d7-4654-b5b0-35bc9822c238", "view", 1000001),
    Row(1624398900,"2021-06-22 23:55:00", "2021-06-22-23", 535871218, "2b1311db-31c8-498b-b5f1-6f9a107580b5", "view", 1000003)
]

prodocut_views_insert_df = spark.createDataFrame(prodocut_views_insert, prodocut_views_schema)
prodocut_views_insert_df.show()

+---------------+-------------------+-------------+---------+--------------------+----------+----------+
|event_timestamp|         event_time|         hour|  user_id|        user_session|event_type|product_id|
+---------------+-------------------+-------------+---------+--------------------+----------+----------+
|     1624397734|2021-06-22 23:35:34|2021-06-22-23|535871217|c6bd7419-2748-4c5...|      view|   1000001|
|     1624398017|2021-06-22 23:40:17|2021-06-22-23|535871217|c6bd7419-2748-4c5...|      view|   1000006|
|     1624398346|2021-06-22 23:45:46|2021-06-22-23|524325294|0b74a829-f9d7-465...|      view|   1000001|
|     1624398650|2021-06-22 23:50:50|2021-06-22-23|524325294|0b74a829-f9d7-465...|      view|   1000001|
|     1624398900|2021-06-22 23:55:00|2021-06-22-23|535871218|2b1311db-31c8-498...|      view|   1000003|
+---------------+-------------------+-------------+---------+--------------------+----------+----------+

In [4]:
prodocuts_data = [
    Row(1000001, "electronics", "smartphone", "apple", 10081.98),
    Row(1000002, "electronics", "tablet", "apple", 2642.17),
    Row(1000003, "electronics", "earphones", "apple", 747.79),
    Row(1000004, "electronics", "earphones", "samsung", 614.53),
    Row(1000005, "electronics", "TV", "samsung", 12347.83),
    Row(1000006, "electronics", "smartphone", "samsung", 7632.76),
    Row(1000007, "electronics", "TV", "sony", 11234.21),
    Row(1000008, "electronics", "earphones", "sony", 398.67)
]


prodocuts_insert_df = spark.createDataFrame(prodocuts_data, producs_schema)
prodocuts_insert_df.show()

+----------+-----------+------------+-------+--------+
|product_id|   category|sub_category|  brand|   price|
+----------+-----------+------------+-------+--------+
|   1000001|electronics|  smartphone|  apple|10081.98|
|   1000002|electronics|      tablet|  apple| 2642.17|
|   1000003|electronics|   earphones|  apple|  747.79|
|   1000004|electronics|   earphones|samsung|  614.53|
|   1000005|electronics|          TV|samsung|12347.83|
|   1000006|electronics|  smartphone|samsung| 7632.76|
|   1000007|electronics|          TV|   sony|11234.21|
|   1000008|electronics|   earphones|   sony|  398.67|
+----------+-----------+------------+-------+--------+

### Create feature groups 

Now We will create each feature group and enable time travel format `HUDI`. In Hopsworks Feature Store `primary` and `partition` keys are required to be privided for HUDI enabled feature groups.

In [5]:
prodocut_views_fg = fs.create_feature_group(
    name = "prodocut_views", 
    description = "E-commerce product views Feature Group",
    version=1,
    primary_key = ["user_id", "event_timestamp"], 
    partition_key = ["hour"], 
    hudi_precombine_key = "user_session", 
    time_travel_format = "HUDI",
    online_enabled=True
)

In [6]:
prodocuts_fg = fs.create_feature_group(
    name = "prodocuts_fg",
    description = "E-commerce producs Feature Group",
    version = 1,
    primary_key = ["product_id"],
    time_travel_format="HUDI",
    online_enabled=True    
)

### Define user provided hudi options 
By default, Hudi tends to over-partition input. Recommended shuffle parallelism for `hoodie.[insert|upsert|bulkinsert].shuffle.parallelism` is atleast input_data_size/500MB

In [7]:
extra_hudi_options = {
    "hoodie.bulkinsert.shuffle.parallelism":"1", 
    "hoodie.insert.shuffle.parallelism":"1", 
    "hoodie.upsert.shuffle.parallelism":"1",
    "hoodie.parquet.compression.ratio":"0.5"
} 

### Bulk insert data into the feature group
Since we have not yet saved any data into newly created feature groups we will use Apache hudi terminology and `Bulk Insert` data. In HSFS its just issuing `save` method.  

In [8]:
prodocuts_fg.save(prodocuts_insert_df,write_options=extra_hudi_options)

In [9]:
prodocut_views_fg.save(prodocut_views_insert_df,write_options=extra_hudi_options)

#### Hopsworks Feature Store Commits

If you thoroughly followed this demo you probably noticed that Hopsworks Feature Store uses Apache Hudi as its time travel engine. Hudi introduces the notion of `commits` which means that it supports certain properties of traditional databases such as single-table transactions, snapshot isolation, atomic upserts and savepoints for data recovery. If an ingestion fails for some reason, no partial results will be written rather the ingestion will be roll-backed. The commit is implemented using atomic `mv` operation in HDFS. 

Currently, feature groups that we created contain only a single commit each as we've just done a single bulk-insert. Lets explore time line of `economy_fg`:

In [10]:
for item in prodocut_views_fg.commit_details().items():
    print(item)

(1624400495000, {'committedOn': '20210622222135', 'rowsUpdated': 0, 'rowsInserted': 5, 'rowsDeleted': 0})

In [11]:
for item in prodocuts_fg.commit_details().items():
    print(item)

(1624400432000, {'committedOn': '20210622222032', 'rowsUpdated': 0, 'rowsInserted': 8, 'rowsDeleted': 0})

### Inspect results

In [12]:
prodocuts_fg.read().show()

+----------+------------+-----------+-------+--------+
|product_id|sub_category|   category|  brand|   price|
+----------+------------+-----------+-------+--------+
|   1000001|  smartphone|electronics|  apple|10081.98|
|   1000002|      tablet|electronics|  apple| 2642.17|
|   1000003|   earphones|electronics|  apple|  747.79|
|   1000004|   earphones|electronics|samsung|  614.53|
|   1000005|          TV|electronics|samsung|12347.83|
|   1000006|  smartphone|electronics|samsung| 7632.76|
|   1000007|          TV|electronics|   sony|11234.21|
|   1000008|   earphones|electronics|   sony|  398.67|
+----------+------------+-----------+-------+--------+

In [13]:
prodocut_views_fg.read().show()

+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|product_id|event_type|  user_id|event_timestamp|         event_time|        user_session|         hour|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|   1000001|      view|524325294|     1624398346|2021-06-22 23:45:46|0b74a829-f9d7-465...|2021-06-22-23|
|   1000001|      view|524325294|     1624398650|2021-06-22 23:50:50|0b74a829-f9d7-465...|2021-06-22-23|
|   1000001|      view|535871217|     1624397734|2021-06-22 23:35:34|c6bd7419-2748-4c5...|2021-06-22-23|
|   1000006|      view|535871217|     1624398017|2021-06-22 23:40:17|c6bd7419-2748-4c5...|2021-06-22-23|
|   1000003|      view|535871218|     1624398900|2021-06-22 23:55:00|2b1311db-31c8-498...|2021-06-22-23|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+

### Upsert new data into a Feature Group

So far we have not done anything time travel special, we simply did a regular bulk-insert of some data into a Hudi enabled feature group. We could have done the same thing using just regular None Hudi enabled Feature group. However now we will look into how we can do upserts, and how Hopsworks Feature store enables us to do this efficiently.

#### Generate Sample Upserts Data

#### Make the Upsert using Hopsworks Feature Store API
In Hopsworks Feature Store issuing `insert` method on Apache Hudi enabled feature group will by default perform `Upsert` operation which means to either insert a new row, or on the basis of `parimary` and `partition` keys update already existing one.

In [14]:
prodocut_views_insert = [
    Row(1624399508, "2021-06-23 00:05:08", "2021-06-23-00", 524325294, "0b74a829-f9d7-4654-b5b0-35bc9822c238", "cart", 1000001),
    Row(1624399846, "2021-06-23 00:10:46", "2021-06-23-00", 543272936, "8187d148-3c41-46d4-b0c0-9c08cd9dc564", "view", 1000006),
    Row(1624400156, "2021-06-23 00:15:56", "2021-06-23-00", 543272936, "a57e52b0-c339-4cc8-8d6f-32fd1dd1df65", "view", 1000006),
    Row(1624399140, "2021-06-22 23:59:00", "2021-06-22-23", 535871218, "2b1311db-31c8-498b-b5f1-6f9a107580b5", "view", 1000003)
]

prodocut_views_insert_df = spark.createDataFrame(prodocut_views_insert, prodocut_views_schema)
prodocut_views_fg.insert(prodocut_views_insert_df,write_options=extra_hudi_options)

In [15]:
prodocut_views_insert = [
    Row(1624400414, "2021-06-23 00:20:14", "2021-06-23-00", 543272936, "8187d148-3c41-46d4-b0c0-9c08cd9dc564", "purchase", 1000006),
    Row(1624400730, "2021-06-23 00:25:30", "2021-06-23-00", 543272936, "a57e52b0-c339-4cc8-8d6f-32fd1dd1df65", "view", 1000006),
    Row(1624401033, "2021-06-23 00:30:33", "2021-06-23-00", 524325294, "0b74a829-f9d7-4654-b5b0-35bc9822c238", "purchase", 1000001),
    Row(1624401348, "2021-06-23 00:35:48", "2021-06-23-00", 543272936, "8187d148-3c41-46d4-b0c0-9c08cd9dc564", "view", 1000006)
]

prodocut_views_insert_df = spark.createDataFrame(prodocut_views_insert, prodocut_views_schema)
prodocut_views_fg.insert(prodocut_views_insert_df,write_options=extra_hudi_options)

In [16]:
prodocut_views_insert = [
    Row(1624401649, "2021-06-23 00:40:49", "2021-06-23-00", 543272936, "a57e52b0-c339-4cc8-8d6f-32fd1dd1df65", "purchase", 1000006),
    Row(1624401902, "2021-06-23 00:45:02", "2021-06-23-00", 543272936, "8187d148-3c41-46d4-b0c0-9c08cd9dc564", "view", 1000006),
    Row(1624402205, "2021-06-23 00:50:05", "2021-06-23-00", 543272936, "a57e52b0-c339-4cc8-8d6f-32fd1dd1df65", "view", 1000006),
    Row(1624402522, "2021-06-23 00:55:22", "2021-06-23-00", 535871217, "c6bd7419-2748-4c56-95b4-8cec9ff8b80d", "cart", 1000002),
    Row(1624402682, "2021-06-23 00:58:02", "2021-06-23-00", 543272936, "a57e52b0-c339-4cc8-8d6f-32fd1dd1df65", "view", 1000006),
    Row(1624402793, "2021-06-23 00:59:53", "2021-06-23-00", 535871217, "c6bd7419-2748-4c56-95b4-8cec9ff8b80d", "purchase", 1000002)
]
        
prodocut_views_insert_df = spark.createDataFrame(prodocut_views_insert, prodocut_views_schema)
prodocut_views_fg.insert(prodocut_views_insert_df,write_options=extra_hudi_options)

#### Inspect the results

Notice that although Hudi enabled Feature group stores the old value of the records from the previous commit, when you query it will only return the values of the latest commit.

In [17]:
prodocut_views_fg.read().show()

+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|product_id|event_type|  user_id|event_timestamp|         event_time|        user_session|         hour|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|   1000001|      cart|524325294|     1624399508|2021-06-23 00:05:08|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624399846|2021-06-23 00:10:46|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624400156|2021-06-23 00:15:56|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000001|  purchase|524325294|     1624401033|2021-06-23 00:30:33|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624400730|2021-06-23 00:25:30|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000006|  purchase|543272936|     1624400414|2021-06-23 00:20:14|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624401348|2021-0

#### Inspect the updated commit timeline of `economyFg`

In [18]:
for item in prodocut_views_fg.commit_details().items():
    print(item)

(1624400675000, {'committedOn': '20210622222435', 'rowsUpdated': 0, 'rowsInserted': 6, 'rowsDeleted': 0})
(1624400617000, {'committedOn': '20210622222337', 'rowsUpdated': 0, 'rowsInserted': 4, 'rowsDeleted': 0})
(1624400552000, {'committedOn': '20210622222232', 'rowsUpdated': 0, 'rowsInserted': 4, 'rowsDeleted': 0})
(1624400495000, {'committedOn': '20210622222135', 'rowsUpdated': 0, 'rowsInserted': 5, 'rowsDeleted': 0})

### Time Travel Queries
When `read` method is issued on `FeatureGroup` object, whithout any aparameters, most recent view of the Feature group will be returned. 

In [19]:
prodocut_views_fg.read().show()

+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|product_id|event_type|  user_id|event_timestamp|         event_time|        user_session|         hour|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|   1000001|      cart|524325294|     1624399508|2021-06-23 00:05:08|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624399846|2021-06-23 00:10:46|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624400156|2021-06-23 00:15:56|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000001|  purchase|524325294|     1624401033|2021-06-23 00:30:33|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624400730|2021-06-23 00:25:30|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000006|  purchase|543272936|     1624400414|2021-06-23 00:20:14|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624401348|2021-0

Using the timeline metadata we can inspect the value of a table at a specific point in time, as well as pull changes incrementally.

In [20]:
for item in prodocut_views_fg.commit_details().items():
    print(item)

commit_timestamps = [prodocut_views_fg.commit_details()[c]['committedOn'] for c in sorted(prodocut_views_fg.commit_details().keys())]

(1624400675000, {'committedOn': '20210622222435', 'rowsUpdated': 0, 'rowsInserted': 6, 'rowsDeleted': 0})
(1624400617000, {'committedOn': '20210622222337', 'rowsUpdated': 0, 'rowsInserted': 4, 'rowsDeleted': 0})
(1624400552000, {'committedOn': '20210622222232', 'rowsUpdated': 0, 'rowsInserted': 4, 'rowsDeleted': 0})
(1624400495000, {'committedOn': '20210622222135', 'rowsUpdated': 0, 'rowsInserted': 5, 'rowsDeleted': 0})

In [21]:
#pull 1st commit
prodocut_views_fg.read(commit_timestamps[0]).show()

+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|product_id|event_type|  user_id|event_timestamp|         event_time|        user_session|         hour|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|   1000001|      view|524325294|     1624398346|2021-06-22 23:45:46|0b74a829-f9d7-465...|2021-06-22-23|
|   1000001|      view|524325294|     1624398650|2021-06-22 23:50:50|0b74a829-f9d7-465...|2021-06-22-23|
|   1000001|      view|535871217|     1624397734|2021-06-22 23:35:34|c6bd7419-2748-4c5...|2021-06-22-23|
|   1000006|      view|535871217|     1624398017|2021-06-22 23:40:17|c6bd7419-2748-4c5...|2021-06-22-23|
|   1000003|      view|535871218|     1624398900|2021-06-22 23:55:00|2b1311db-31c8-498...|2021-06-22-23|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+

In [22]:
#pull 2nd commit
prodocut_views_fg.read(commit_timestamps[1]).show()

+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|product_id|event_type|  user_id|event_timestamp|         event_time|        user_session|         hour|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|   1000001|      cart|524325294|     1624399508|2021-06-23 00:05:08|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624399846|2021-06-23 00:10:46|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624400156|2021-06-23 00:15:56|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000001|      view|524325294|     1624398346|2021-06-22 23:45:46|0b74a829-f9d7-465...|2021-06-22-23|
|   1000001|      view|524325294|     1624398650|2021-06-22 23:50:50|0b74a829-f9d7-465...|2021-06-22-23|
|   1000001|      view|535871217|     1624397734|2021-06-22 23:35:34|c6bd7419-2748-4c5...|2021-06-22-23|
|   1000006|      view|535871217|     1624398017|2021-0

In [23]:
#pull 3rd commit
prodocut_views_fg.read(commit_timestamps[2]).show()

+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|product_id|event_type|  user_id|event_timestamp|         event_time|        user_session|         hour|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|   1000001|      cart|524325294|     1624399508|2021-06-23 00:05:08|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624399846|2021-06-23 00:10:46|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624400156|2021-06-23 00:15:56|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000001|  purchase|524325294|     1624401033|2021-06-23 00:30:33|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624400730|2021-06-23 00:25:30|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000006|  purchase|543272936|     1624400414|2021-06-23 00:20:14|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624401348|2021-0

### Hopsworks Feature Store also provides a method for incremental reads:

In [24]:
#Pull changes that happened between the first and second commits
prodocut_views_fg.read_changes(commit_timestamps[0], commit_timestamps[1]).show()

+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|product_id|event_type|  user_id|event_timestamp|         event_time|        user_session|         hour|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|   1000001|      cart|524325294|     1624399508|2021-06-23 00:05:08|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624399846|2021-06-23 00:10:46|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624400156|2021-06-23 00:15:56|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000003|      view|535871218|     1624399140|2021-06-22 23:59:00|2b1311db-31c8-498...|2021-06-22-23|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+

In [25]:
prodocut_views_fg.read_changes(commit_timestamps[0], commit_timestamps[1]).show()

+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|product_id|event_type|  user_id|event_timestamp|         event_time|        user_session|         hour|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|   1000001|      cart|524325294|     1624399508|2021-06-23 00:05:08|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624399846|2021-06-23 00:10:46|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624400156|2021-06-23 00:15:56|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000003|      view|535871218|     1624399140|2021-06-22 23:59:00|2b1311db-31c8-498...|2021-06-22-23|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+

In [26]:
#Pull changes that happened between the first and third commits 
prodocut_views_fg.read_changes(commit_timestamps[0], commit_timestamps[2]).show()

+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|product_id|event_type|  user_id|event_timestamp|         event_time|        user_session|         hour|
+----------+----------+---------+---------------+-------------------+--------------------+-------------+
|   1000001|      cart|524325294|     1624399508|2021-06-23 00:05:08|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624399846|2021-06-23 00:10:46|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624400156|2021-06-23 00:15:56|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000001|  purchase|524325294|     1624401033|2021-06-23 00:30:33|0b74a829-f9d7-465...|2021-06-23-00|
|   1000006|      view|543272936|     1624400730|2021-06-23 00:25:30|a57e52b0-c339-4cc...|2021-06-23-00|
|   1000006|  purchase|543272936|     1624400414|2021-06-23 00:20:14|8187d148-3c41-46d...|2021-06-23-00|
|   1000006|      view|543272936|     1624401348|2021-0

## Perform incremental descriptive statistics: count type of events that occured between 1st and 4th commit 

In [27]:
prodocut_views_fg.read_changes(commit_timestamps[0], commit_timestamps[3]).select("event_type").groupBy('event_type').count().show()

+----------+-----+
|event_type|count|
+----------+-----+
|  purchase|    4|
|      view|    8|
|      cart|    2|
+----------+-----+

### Join Feature groups that correspond to specific point in time
If we are interetsted to join Feature groups all of them correspong to one specific point in time then we can issue `as_of` method on join `Query` object.  

In [28]:
joined_features = ((prodocuts_fg.select_all())
                   .join(prodocut_views_fg.select_all(), ["product_id"], "INNER", prefix="views_")
                   .as_of(commit_timestamps[2]))  

In [29]:
joined_features.read().show()

+----------+------------+-----------+-------+--------+----------------+----------------+-------------+---------------------+-------------------+--------------------+-------------+
|product_id|sub_category|   category|  brand|   price|views_product_id|views_event_type|views_user_id|views_event_timestamp|   views_event_time|  views_user_session|   views_hour|
+----------+------------+-----------+-------+--------+----------------+----------------+-------------+---------------------+-------------------+--------------------+-------------+
|   1000001|  smartphone|electronics|  apple|10081.98|         1000001|            cart|    524325294|           1624399508|2021-06-23 00:05:08|0b74a829-f9d7-465...|2021-06-23-00|
|   1000001|  smartphone|electronics|  apple|10081.98|         1000001|        purchase|    524325294|           1624401033|2021-06-23 00:30:33|0b74a829-f9d7-465...|2021-06-23-00|
|   1000001|  smartphone|electronics|  apple|10081.98|         1000001|            view|    52432529

### Join Feature groups that correspond to different points in time
Hopswork Feature store also provides functionality to join Feature groups that correspond to different points in time. 

In [30]:
prodocuts_fg_query = prodocuts_fg.select_all().as_of(commit_timestamps[0])
prodocut_views_fg_query =  prodocut_views_fg.select_all().as_of(commit_timestamps[2])

In [31]:
joined_features = prodocuts_fg_query.join(prodocut_views_fg_query, ["product_id"], "INNER", prefix="views_")

In [32]:
joined_features.read().show()

+----------+------------+-----------+-------+--------+----------------+----------------+-------------+---------------------+-------------------+--------------------+-------------+
|product_id|sub_category|   category|  brand|   price|views_product_id|views_event_type|views_user_id|views_event_timestamp|   views_event_time|  views_user_session|   views_hour|
+----------+------------+-----------+-------+--------+----------------+----------------+-------------+---------------------+-------------------+--------------------+-------------+
|   1000001|  smartphone|electronics|  apple|10081.98|         1000001|            cart|    524325294|           1624399508|2021-06-23 00:05:08|0b74a829-f9d7-465...|2021-06-23-00|
|   1000001|  smartphone|electronics|  apple|10081.98|         1000001|        purchase|    524325294|           1624401033|2021-06-23 00:30:33|0b74a829-f9d7-465...|2021-06-23-00|
|   1000001|  smartphone|electronics|  apple|10081.98|         1000001|            view|    52432529